In [1]:
from adversarial_lab.core import ALModel
from adversarial_lab.core.noise_generators import AdditiveNoiseGenerator
from adversarial_lab.attacks.whitebox import WhiteBoxMisclassification
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.utils.plotting import plot_images_and_noise

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [2]:
model = InceptionV3(weights='imagenet')


def preprocess(data, *args, **kwargs):
    input = tf.cast(data, dtype=tf.float32)
    if len(input.shape) == 2:
        input = tf.expand_dims(input, axis=-1)
        input = tf.image.grayscale_to_rgb(input)

    elif len(input.shape) == 3 and input.shape[-1] == 1:
        input = tf.image.grayscale_to_rgb(input)

    input_tensor = tf.convert_to_tensor(input, dtype=tf.float32)
    resized_image = tf.image.resize(input_tensor, (299, 299))
    batch_image = tf.expand_dims(resized_image, axis=0)
    return preprocess_input(batch_image)


pp = PreprocessingFromFunction.create(preprocess)

In [3]:
image = Image.open('data/panda.jpg')
image_array = np.array(image)

np.argmax(model.predict(pp.preprocess(image_array)), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([388], dtype=int64)

In [4]:
from adversarial_lab.analytics import AdversarialAnalytics, ImageTracker, LossTracker, PredictionsTracker
from adversarial_lab.db.sqllite import SqlliteDB

db = SqlliteDB(db_path="whitebox.db")
analytics = AdversarialAnalytics(db,
                                 [
                                     LossTracker(track_batch=False),
                                     ImageTracker(),
                                     PredictionsTracker()
                                 ],
                                 "test",
                                 True)

In [5]:
from adversarial_lab.core.losses import CategoricalCrossEntropy
from adversarial_lab.core.penalties import LpNorm
from adversarial_lab.core.optimizers import PGD

optim = PGD(learning_rate=0.0001)

loss = CategoricalCrossEntropy(penalties=[LpNorm(p=2, lambda_val=0.3)])

In [6]:
attacker = WhiteBoxMisclassification(
    model, optim, loss, preprocessing=pp, analytics=analytics, verbose=3)

In [7]:
noise = attacker.attack(image_array, target_class=924,
                        epochs=200, strategy="random")

Attacking:   0%|          | 0/200 [00:00<?, ?it/s]

Attacking:  20%|█▉        | 39/200 [00:34<02:17,  1.17it/s, Loss=15.3, Prediction (score)=388(0.582), Original Class (score)=388(0.582), Target Class (score)=924(0.002)]

KeyboardInterrupt: 

In [ ]:
plot_images_and_noise(pp.preprocess(image_array), noise[0])